In [ ]:
import os
import cv2
import numpy as np


In [ ]:
titles={}
frames={}
audios={}
visuals={}
objects={}

In [8]:
import pickle
fol=os.listdir('/Users/macbookpro2017/Desktop/UnsupervisedVideoSummarization/video_ext_data')

data=[]
for videoID in fol:
    files=os.listdir('/Users/macbookpro2017/Desktop/UnsupervisedVideoSummarization/video_ext_data/'+videoID)

    for file in files:
        if('model' not in file):
            with open('/Users/macbookpro2017/Desktop/UnsupervisedVideoSummarization/video_ext_data/'+videoID+'/'+file,'rb') as f:
                data=pickle.load(f)
                if('title' in f.name):
                    titles[videoID]=(data)
                elif ('audio' in f.name):
                    audios[videoID]=(data)
                elif ('visual' in f.name):
                    visuals[videoID]=(data)
                elif ('objects' in f.name):
                    objects[videoID]=(data)


In [10]:
import pandas as pd
import os
import numpy as np
from math import ceil,floor

def calculate_mean_annotations(dir_anno, video_folder):
    # Load annotations
    df = pd.read_csv(dir_anno, sep='\t', header=None, names=['videoID', 'category', 'annotations'])

    # Get list of video IDs
    video_ids = [f for f in os.listdir(video_folder)]
    # Prepare result list
    results = {}

    # Calculate mean annotations for each video
    for videoid in video_ids:
        # Filter DataFrame for current video ID
        video_df = df[df['videoID'] == videoid]

        # If no annotations for this video, skip
        if video_df.empty:
            continue

        # Process each frame
        frame_means = []
        for _, row in video_df.iterrows():
            # Split the annotation string and convert to numeric
            frame_means.append([float(a) for a in row['annotations'].split(',')])
            
        # Calculate mean of all annotators for this frame
        frame_means=(np.array(frame_means).mean(axis=0)) #mean of all annotators for each frame
        
        # Round the mean to the nearest integer
        for index,frame in enumerate(frame_means):
            if abs(frame-ceil(frame))>0.5:
                frame_means[index]=floor(frame)
            else:
                frame_means[index]=ceil(frame)

        # Store in results
        results[videoid] = frame_means

    return results

# Example usage
dir_anno = 'datasets/ydata-tvsum50-v1_1/data/ydata-tvsum50-anno.tsv'
video_folder = '/Users/macbookpro2017/Desktop/UnsupervisedVideoSummarization/video_ext_data'
mean_annotations = calculate_mean_annotations(dir_anno, video_folder)


In [11]:
from sklearn.preprocessing import OneHotEncoder
def one_hot_encode(objects_in_frame, unique_objects):
    # Handle None or empty values as 'no_object'
    object_array = ['no_object' if x is None or len(x) == 0 else x for x in objects_in_frame]

    # Reshape the array for OneHotEncoder
    object_array = np.array(object_array).reshape(-1, 1)

    # Initialize OneHotEncoder
    encoder = OneHotEncoder(sparse_output=False,dtype=float, categories=[unique_objects])

    # Fit and transform the data
    one_hot_encoded = encoder.fit_transform(object_array)
    return one_hot_encoded

def detectObjects(objects):
        
    # Here, taking the first detected object
    objects = [frame_objects[0] if frame_objects else 'None' for frame_objects in objects]

    # Generate unique objects and include 'no_object'
    unique_objects = sorted(set(objects + ['no_object']))

    # One-hot encoding of objects
    encoded_objects = one_hot_encode(objects, unique_objects)

    # Padding not required as one-hot encoding ensures consistent vector length
    
    return encoded_objects

In [12]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

def train_lstm_model(combined_data_all, labels_all):
    # Pad the sequences for LSTM
    max_length = max(len(feature) for feature in combined_data_all)
    padded_features = pad_sequences(combined_data_all, maxlen=max_length, padding='post', dtype='float32')

    # Reshape labels if necessary
    labels_all = np.array(labels_all)

    # Define LSTM model
    model = Sequential()
    model.add(LSTM(50, input_shape=(max_length, padded_features.shape[2])))
    model.add(Dense(1, activation='sigmoid'))  # Modify based on your problem

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])  # Modify as needed

    print('FIT MODEL STARTS')
    # Train the model
    model.fit(padded_features, labels_all, epochs=10, batch_size=32)

    # Save the model
    model.save('lstm_model.h5')

    return model


2023-12-04 22:27:44.343814: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [13]:

import numpy as np
from joblib import dump


combined_data_all = []
labels_all = []

# Combine all data
for key, item in mean_annotations.items():
    title = titles[key]
    audio = audios[key]
    visual = visuals[key]
    object = objects[key]
    
    padded_encoded_objects = detectObjects(object)
    combined_data=[]
    for i, frame in enumerate(audio):
        combined_feature = np.concatenate([
            np.array(visual[i], dtype=float),
            np.array(audio[i], dtype=float),
            np.array(padded_encoded_objects[i], dtype=float),
            np.array(title, dtype=float).reshape(-1),
        ])    
        combined_data_all.append(combined_feature)
        labels_all.append(item[i])
        
        
# Train LSTM model
lstm_model = train_lstm_model(combined_data_all, labels_all)


print("LSTM Complete.")

# Save the trained SVM model
dump(lstm_model, 'model/trained_lstm_model.joblib')

print("Model training complete and saved.")

IndexError: tuple index out of range